In [29]:
from markitdown import MarkItDown
import os 
from PyPDF2 import PdfFileReader, PdfFileWriter 
from pypdf import PdfReader, PdfWriter
import shlex
import requests
import re

Garder la dernière page du PDF : 

In [ ]:
def extract_last_page(input_path, output_path):
    # On lit le PDF original
    reader = PdfReader(input_path)
    writer = PdfWriter()

    # On garde la dernière page
    last_page = reader.pages[-1]
    writer.add_page(last_page)

    # On la met dans un nouveau PDF
    with open(output_path, "wb") as output_file:
        writer.write(output_file)

    print(f"Last page saved to: {output_path}")

In [ ]:
# on vérifie que le fichier n'est pas html (c'est le cas quand c'est mal extrait)

def is_html(file_path):
    with open(file_path, "rb") as f:
        header = f.read(1024).lower()
        return b"<html" in header or b"<!doctype html" in header

Conversion PDF en Markdown

In [15]:
markitdown = MarkItDown()
result = markitdown.convert('pdfs\ACSA%201_2.pdf')
print(result.text_content)

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\lisef\AppData\Local\Temp\ipykernel_16024\700113543.py:2: SyntaxWarning: invalid escape sequence '\A'
  result = markitdown.convert('pdfs\ACSA%201_2.pdf')


ACSA 1

CMA CGM weekly fixed-day service connecting Far East and West Coast South America

Busan

Yokohama

Busan

Yokohama

Shanghai
Ningbo

Shanghai
Ningbo

Keelung

Chiwan

Hong Kong

Manzanillo

Chancay
Callao

Arica

Iquique

INCA

Chancay
Callao

San Antonio
Lirquen

INCA2

Balboa

Posorja

Chancay
Callao
Callao

San Antonio

CMA CGM Strengths

•  Direct and unique product offering fastest transit time between Far East & West Coast South America (WCSA)
•  Direct call to San Antonio, offering the most competitive transit time on the market
•  Seamless connection with CMA CGM direct fixed-day weekly INCA services (to connect Arica/Iquique/Lirquen)
•  Designed for perishable and fresh fruit in reefer containers from West Coast South America to the Far East

ACSA 1

CMA CGM weekly fixed-day service connecting Far East and West Coast South America

TRANSIT TIMES*

EASTBOUND

SAN ANTONIO

CHANCAY

CALLAO

POSORJA

YOKOHAMA

BUSAN

SHANGHAI

NINGBO

41

36

32

29

WESTBOUND

49

45

4

LLM - PDF : 

In [ ]:
# Extraction des tableaux à l'aide de la plateforme de LLM

input_file_1 = "pdfs\WAX_0.pdf"

with open(input_file_1, "r", encoding="utf-8") as f1:
    pdf_content = PdfReader('C:\\Users\\lisef\\projet-buyco\\Buyco\\pdfs\\WAX_0.pdf').pages[0]

input_file_2 = "WAX_0.md"

with open(input_file_2, "r", encoding="utf-8") as f2:
    markdown_content = f2.read()


prompt_LLM_fra = f"""Voici le contenu d'un fichier PDF pour comprendre la forme et d'un fichier Markdown. 
Extrait uniquement les tableaux de temps de transit** (Transit Times), et rends-les sous forme de 
tableaux lisibles .csv Ne commente pas, ne reformule pas.
PDF : {pdf_content}, Markdown : {markdown_content}"""

prompt_LLM = f"""This is a PDF file to understand the way it is and a Markdown file. Extract only 
Transit Times' tables and give them back to me on a readable .csv format. Don't comment and don't repharse.
PDF : {pdf_content}, Markdown : {markdown_content}"""

safe_prompt_LLM = shlex.quote(prompt_LLM)

output_file = "WAX_0.txt"


# Méthode requests

url = "http://ollama-sam.inria.fr/api/generate"
data = {'model': 'mistral:7b', 'prompt': safe_prompt_LLM}
auth = ('Bob', 'hiccup')  # HTTP Basic Auth

response = requests.post(url, json=data, auth=auth)


with open(output_file, "w", encoding="utf-8") as out_file:
    out_file.write(response.text)



# Extraitre les fichiers .csv du fichier text

<>:3: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
C:\Users\lisef\AppData\Local\Temp\ipykernel_16024\2788557382.py:3: SyntaxWarning: invalid escape sequence '\W'
  input_file_1 = "pdfs\WAX_0.pdf"


Boucler sur tous les fichiers

In [35]:
import pdfplumber
import shlex
import requests
import re
import json
import csv
from pypdf import PdfReader, PdfWriter

path = 'C:\\Users\\lisef\\projet-buyco\\Buyco\\pdfs_extraits' # chemin d'accès des pdfs
subdirectories = os.listdir(path)

for fichier in subdirectories:
    if not is_html(f'C:\\Users\\lisef\\projet-buyco\\Buyco\\pdfs_extraits\\{fichier}'):
        vrai_fichier = re.search('[^.]*',f'{fichier}')
        nom_fichier = vrai_fichier.group(0) # on ne garde que le nom sans l'extension

        # Conversion pdf en Markdown

    input_file = f"pdfs_extraits\{nom_fichier}.pdf" 
    output_file = f"{nom_fichier}.md" 

    def pdf_tables_to_markdown(pdf_path, output_md):
        with pdfplumber.open(pdf_path) as pdf, open(output_md, "w", encoding="utf-8") as f:
            for page_num, page in enumerate(pdf.pages, 1):
                tables = page.extract_tables()
                print(f"Page {page_num}: {len(tables)} tables détectées")
                for i, table in enumerate(tables, 1):
                    if not table or not table[0]:
                        continue  
                    md = ""
                    md += "| " + " | ".join(cell if cell is not None else "" for cell in table[0]) + " |\n"
                    md += "| " + " | ".join(["---"] * len(table[0])) + " |\n"
                    for row in table[1:]:
                        md += "| " + " | ".join(cell if cell else "" for cell in row) + " |\n"
                    md += "\n\n" 
                    f.write(md)

    pdf_tables_to_markdown(input_file, output_file)



    # Extraction des tableaux à l'aide de la plateforme de LLM

    input_file = f"{nom_fichier}.md"
    output_file = f"{nom_fichier}.txt" 

    with open(input_file, "r", encoding="utf-8") as f:
        markdown_content = f.read()


    prompt_LLM = f"""This is a Markdown file. Extract only the tables and give them back to me on a readable
    .csv format. Don't comment and don't repharse. Above all else, dont add information from outside the Markdown
    file. Let the tables how they are in the Markdown, do not mix them. When there is a line feed, it changes tables, so do it. There are always tables.
    Often (but not always) a table starts with 'from' 'to'; Markdown : {markdown_content}""" #Transit Times'
    safe_prompt_LLM = shlex.quote(prompt_LLM)

    url = "http://ollama-sam.inria.fr/api/generate"
    data = {'model': 'mistral:7b', 'prompt': safe_prompt_LLM}
    auth = ('Bob', 'hiccup')  # HTTP Basic Auth

    response = requests.post(url, json=data, auth=auth)

    with open(output_file, "w", encoding="utf-8") as out_file:
        out_file.write(response.text)




    # Traitement du fichier .txt donné en réponse par le LLM

    input_file = f"{nom_fichier}.txt"
    response_file = f"{nom_fichier}_response.txt"
    output_file = f"{nom_fichier}_table.csv"

    responses = []
    with open(input_file, "r", encoding="utf-8") as text_file:
        for line in text_file:
            try:
                obj = json.loads(line.strip())
                if "response" in obj:
                    responses.append(obj["response"])
            except json.JSONDecodeError:
                continue

    full_text = "".join(responses)
    full_text = full_text.replace('"""', '').replace('"', '')

    with open(response_file, "w", encoding="utf-8") as file:
        file.write(full_text)


    # Conversion du fichier .txt amélioré en fichier .csv
    lines = [line.strip() for line in full_text.strip().splitlines() if line.strip()]

    with open(output_file, "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        for line in lines:
            writer.writerow(line.split(","))

    # Vérification que le model LLM n'invente pas des données et fais simplement de l'extraction de données
    # Faire un test de vérification

    input_file = f"{nom_fichier}_response.txt"
    output_base = f"{nom_fichier}_table"

    with open(input_file, "r", encoding="utf-8") as f:
        content = f.read()

    # Extraire contenu entre ```
    match = re.search(r"```(.*?)```", content, re.DOTALL)
    if not match:
        raise ValueError("Bloc markdown ``` non trouvé dans le fichier de réponse")

    tables_text = match.group(1).strip()

    lines = [line.strip() for line in tables_text.splitlines() if line.strip()]

    # Trouver l'index où la deuxième table commence
    split_index = None
    prev_len = None
    for i, line in enumerate(lines):
        cols = line.split(",")
        if prev_len is not None and len(cols) > prev_len:
            split_index = i
            break
        prev_len = len(cols)

    if split_index is None:
        tables = [lines]
    else:
        tables = [lines[:split_index], lines[split_index:]]
    if len(tables) > 1:
        second_table = tables[1]
        first_line_cols = second_table[0].split(",")
        first_line_cols[0] = "To\nFrom"
        import io
        output = io.StringIO()
        writer = csv.writer(output, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(first_line_cols)
        tables[1][0] = output.getvalue().strip() 

    # Écriture des tables dans des fichiers CSV
    for i, table_lines in enumerate(tables, 1):
        output_file = f"{output_base}_{i}.csv"
        with open(output_file, "w", newline="", encoding="utf-8") as csv_file:
            writer = csv.writer(csv_file, quoting=csv.QUOTE_MINIMAL)
            for line in table_lines:
                row = next(csv.reader([line]))
                writer.writerow(row)
        print(f"Table {i} saved in {output_file}")

<string>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\{'
<string>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\{'
C:\Users\lisef\AppData\Local\Temp\ipykernel_16024\700528354.py:19: SyntaxWarning: invalid escape sequence '\{'
  input_file = f"pdfs_extraits\{nom_fichier}.pdf"


Page 1: 3 tables détectées
Table 1 saved in extract_ABIDJAN%20REEFER%20TUNA%20EXPORT_table_1.csv
Table 2 saved in extract_ABIDJAN%20REEFER%20TUNA%20EXPORT_table_2.csv
Page 1: 6 tables détectées
Table 1 saved in extract_ACSA%201_2_table_1.csv
Table 2 saved in extract_ACSA%201_2_table_2.csv
Page 1: 6 tables détectées
Table 1 saved in extract_ACSA%202_2_table_1.csv
Table 2 saved in extract_ACSA%202_2_table_2.csv
Page 1: 6 tables détectées
Table 1 saved in extract_ACSA%203_0_table_1.csv
Table 2 saved in extract_ACSA%203_0_table_2.csv
Page 1: 2 tables détectées


C:\Users\lisef\AppData\Local\Temp\ipykernel_16024\700528354.py:19: SyntaxWarning: invalid escape sequence '\{'
  input_file = f"pdfs_extraits\{nom_fichier}.pdf"


ValueError: Bloc markdown ``` non trouvé dans le fichier de réponse